In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, MaxPool2D, Conv2D, Flatten, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import cv2
%matplotlib inline
%config Completer.use_jedi = False
%config IPCompleter.use_jedi = False

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=.2, zoom_range=.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
training_set = train_datagen.flow_from_directory("../input/chest-xray-pneumonia/chest_xray/train", target_size=(150,150), batch_size=32, class_mode="binary")
testing_set = test_datagen.flow_from_directory("../input/chest-xray-pneumonia/chest_xray/test", target_size=(150,150), batch_size=32, class_mode="binary")

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [4]:
es = EarlyStopping(monitor='val_accuracy', mode="max", patience=25, verbose=1, restore_best_weights=True)
cp = ModelCheckpoint("Weights-{epoch:03d}--{val_accuracy:.5f}.hdf5", monitor='val_accuracy', verbose = 2, save_best_only = True, mode ='max')
rl = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1, factor=0.3, min_lr=0.000001)

In [5]:
model = Sequential()
model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (150,150,3)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 128 , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.compile(optimizer = "rmsprop" , loss = 'binary_crossentropy' , metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 150, 150, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 75, 75, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 38, 64)        0

In [6]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [7]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [8]:
log = model.fit(x = training_set, validation_data = testing_set, batch_size=16, epochs=200, callbacks=[cp, es, rl])
model.save("model.hdf5")
model.summary()

Epoch 1/200
163/163 [==============================] - 133s 789ms/step - loss: 1.1204 - accuracy: 0.8056 - val_loss: 4.5336 - val_accuracy: 0.6250

Epoch 00001: val_accuracy improved from -inf to 0.62500, saving model to Weights-001--0.62500.hdf5
Epoch 2/200
163/163 [==============================] - 80s 491ms/step - loss: 0.2906 - accuracy: 0.9042 - val_loss: 2.2301 - val_accuracy: 0.7035

Epoch 00002: val_accuracy improved from 0.62500 to 0.70353, saving model to Weights-002--0.70353.hdf5
Epoch 3/200
163/163 [==============================] - 83s 511ms/step - loss: 0.2070 - accuracy: 0.9255 - val_loss: 6.8267 - val_accuracy: 0.6250

Epoch 00003: val_accuracy did not improve from 0.70353
Epoch 4/200
163/163 [==============================] - 80s 488ms/step - loss: 0.1758 - accuracy: 0.9356 - val_loss: 0.3840 - val_accuracy: 0.8686

Epoch 00004: val_accuracy improved from 0.70353 to 0.86859, saving model to Weights-004--0.86859.hdf5
Epoch 5/200
163/163 [==============================] 

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [9]:
images = []
name = []
folder_path = '../input/chest-xray-pneumonia/chest_xray/test/NORMAL'
for img in os.listdir(folder_path):
    name.append(os.path.join(folder_path, img))
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(150, 150))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img)
    
images = np.vstack(images)
classes = model.predict_classes(images, batch_size=10)
for i in classes:
    if i[0] == 0:
        i[0] = "Normal"
    else:
        i[0] = "Pneumonia"
classes

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


ValueError: invalid literal for int() with base 10: 'Normal'

In [10]:
images = []
name = []
folder_path = '../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA'
for img in os.listdir(folder_path):
    name.append(os.path.join(folder_path, img))
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(150, 150))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img)
    
images = np.vstack(images)
classes = model.predict_classes(images, batch_size=10)
classes

array([[1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1]], dtype=int32)